In [52]:
import json
import re
import core.utils as oa
from rapidfuzz import fuzz
import pandas as pd
import io
import os

import datetime
from numpyencoder import NumpyEncoder
from pathlib import Path

# root directory path
ROOT = Path(os.getcwd()).resolve().parents[0]

In [53]:
date = datetime.datetime.now().strftime("%y-%m-%d_%H:%M")
date

'25-09-20_14:12'

In [54]:
print("Sollen Bibelstellen (1) oder Liedzitate (2) gesucht werden?")
response = None
while response not in ["1", "2"]:
    response = input("Bitte Nummer eingeben")

if response == "1":
    task = "bibel"
    print("Bibelstellen werden gesucht")
else:
    task = "lieder"
    print("Liedzitate werden gesucht")


Sollen Bibelstellen (1) oder Liedzitate (2) gesucht werden?
Bibelstellen werden gesucht


In [55]:
print("Sollen Predigten mit den meisten (1), oder den längsten (2) Liedzitaten analysiert werden?")
response = None
while response not in ["1", "2"]:
    response = input("Bitte Nummer eingeben")

if response == "2":
    corpus = "longest"
else:
    corpus = "most"


with open(ROOT / f"sermons_with_{corpus}_music.json", "r") as f:
    testsermons = json.load(f)

Sollen Predigten mit den meisten (1), oder den längsten (2) Liedzitaten analysiert werden?


In [56]:
relevant_page_texts = []
if task == "lieder":
    for n in range(41, 1291):
        with open(ROOT / f"source_texts/praxis_pietatis_verses/{n}.json") as f:
            page = json.load(f)
        page_info = {}
        page_info[n] = page
        relevant_page_texts.append(page_info)
else:
    with open(ROOT / "source_texts/bible/old_testament_chunked.json") as f:
        old_testament = json.load(f)
    for book_name, chapters in old_testament.items():
        for chapter, verses in chapters.items():
            for verse_nr, verse in verses.items():
                verse_info = {}
                verse_id = f"{book_name}_{chapter}_{verse_nr}"
                verse_info[verse_id] = verse
                relevant_page_texts.append(verse_info)
    with open(ROOT / f"source_texts/bible/new_testament_chunked.json") as f:
        new_testament = json.load(f)
    for book_name, chapters in old_testament.items():
        for chapter, verses in chapters.items():
            for verse_nr, verse in verses.items():
                verse_info = {}
                verse_id = f"{book_name}_{chapter}_{verse_nr}"
                verse_info[verse_id] = verse
                relevant_page_texts.append(verse_info)

In [57]:
fuzziness = 80

In [58]:
similarity_table = {}
similarity_table["task"] = task
similarity_table['date'] = date
similarity_table['corpus'] = corpus
similarity_table['method'] = 'similarity_search'
similarity_table['fuzziness'] = fuzziness


In [59]:
testsermons = ["E000036"]

In [60]:
import core.similaritysearch as simsearch

similarity_table['results'] = simsearch.find_similarities(task, testsermons, relevant_page_texts, 80)


Starting with E000036


In [ ]:
#with open(ROOT / f'similarity_tables/sim-search_{task}_{corpus}_{fuzziness}_{date}.json', "w") as f:
#    json.dump(similarity_table, f, ensure_ascii=False)

In [61]:
similarity_table

{'task': 'bibel',
 'date': '25-09-20_14:12',
 'corpus': 'most',
 'method': 'similarity_search',
 'fuzziness': 80,
 'results': [['E000036',
   1733,
   'Predigt,Paragraph,Satz,Bibelstelle,Bibelvers,Ähnlichkeit,Dopplung\n0,so frage ich nichts nach himmel und erden,21,15,Psalm_73_25,so frage ich nichts nach Himel vnd Erden,91.36,False\n1,so frage ich nichts nach himmel und erden,21,15,Psalm_73_25,so frage ich nichts nach Himel vnd Erden,91.36,True\n2,wenn mir gleich leib und seel verschmacht,21,16,Psalm_73_26,Wenn mir gleich Leib vnd Seele verschmacht,89.16,False\n3,wenn mir gleich leib und seel verschmacht,21,16,Psalm_73_26,Wenn mir gleich Leib vnd Seele verschmacht,89.16,True\n4,der wird nimmermehr sterben,23,8,Psalm_10_11,Er wirds nimer mehr sehen,80.77,False\n5,der wird nimmermehr sterben,23,8,Psalm_10_11,Er wirds nimer mehr sehen,80.77,True\n6,der mich gesandt hat,23,15,1 Mose_16_14,der mich angesehen hat,80.95,True\n7,der mich gesandt hat,23,15,5 Mose_32_18,der dich gemacht hat,80.0

In [62]:
buffer = io.StringIO(similarity_table['results'][0][2])
df2 = pd.read_csv(buffer)
df2

,Predigt,Paragraph,Satz,Bibelstelle,Bibelvers,Ähnlichkeit,Dopplung
0,so frage ich nichts nach himmel und erden,21,15,Psalm_73_25,so frage ich nichts nach Himel vnd Erden,91.36,False
1,so frage ich nichts nach himmel und erden,21,15,Psalm_73_25,so frage ich nichts nach Himel vnd Erden,91.36,True
2,wenn mir gleich leib und seel verschmacht,21,16,Psalm_73_26,Wenn mir gleich Leib vnd Seele verschmacht,89.16,False
3,wenn mir gleich leib und seel verschmacht,21,16,Psalm_73_26,Wenn mir gleich Leib vnd Seele verschmacht,89.16,True
4,der wird nimmermehr sterben,23,8,Psalm_10_11,Er wirds nimer mehr sehen,80.77,False
...,...,...,...,...,...,...,...
393,und hast lust in demselben zu wohnen,163,12,Psalm_132_13,Vnd hat lust daselbs zu wonen,80.00,True
394,wer ihr achtet der hat eitel lust daran,163,31,Psalm_111_2,Wer jr achtet der hat eitel lust dran,92.11,False
395,wer ihr achtet der hat eitel lust daran,163,31,Psalm_111_2,Wer jr achtet der hat eitel lust dran,92.11,True
396,und deinen nahmen rühmen immer und ewiglich,163,109,Psalm_145_2,Vnd deinen Namen rhümen jmer vnd ewiglich,85.71,False


In [63]:
def get_results_csv(list_of_lists, id):
  for sublist in list_of_lists:
    if len(sublist) >= 3 and sublist[0] == id:
      return sublist[2]
  return None

In [64]:
for id in testsermons:
    # get results for this specific sermon from results tables
    results_csv = get_results_csv(similarity_table['results'], id)
    if results_csv:
        # write results to csv
        filepath = ROOT / f"predictions/{id}_{similarity_table['method']}_{similarity_table['fuzziness']}_{similarity_table['date']}.csv"
        with open(filepath, "w") as f:
            f.write(results_csv)
        
        # collect metadata
        info = {}
        info["date"] = similarity_table['date']
        info["task"] = similarity_table['task']
        info["method"] = similarity_table['method']
        info["fuzziness"] = similarity_table['fuzziness']
        info['file'] = str(filepath)

        # append metadata if file already exists,
        # otherwise create new file
        my_file = Path(ROOT / f"predictions/{id}_predictions.json")
        if my_file.is_file():
            with open(my_file, "r", encoding="utf-8") as f:
                predictions = json.load(f)
            predictions.append(info)
            with open(my_file, "w", encoding="utf-8") as f:
                json.dump(predictions, f, ensure_ascii=False)
        else:
            with open(ROOT / f"predictions/{id}_predictions.json", "x", encoding="utf-8") as f:
                json.dump([info], f, ensure_ascii=False)